In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook
import pytz
import xarray as xr

In [ ]:
# Define a function to parse excel files and return a dataframe
def parse_excel(file_path):
    wb = load_workbook(file_path, read_only=True,data_only=True)
    ws = wb.active

    # Get the date from cell A1
    date = pd.to_datetime(ws['A1'].value, format='%Y%m%d')

    # Create an empty DataFrame
    df = pd.DataFrame()

    # Extract the data from the specified cells
    for row in range(6, 30):
        hour = ws['A' + str(row)].value
        datetime_index = pd.to_datetime(date + pd.DateOffset(hours=hour))
        
        demand_data = ws['B' + str(row)].value
        pv_data = ws['E' + str(row)].value
        mgp_sicily_forecast = ws['H' + str(row)].value
        mgp_sicily_realised = ws['I' + str(row)].value
        mgp_sicily_error = ws['J' + str(row)].value

        data = {'demand_data': [demand_data], 
                'pv_data': [pv_data], 
                'mgp_sicily_forecast': [mgp_sicily_forecast], 
                'mgp_sicily_realised': [mgp_sicily_realised],
                'mgp_sicily_error': [mgp_sicily_error]}

        df_temp = pd.DataFrame(data, index=[datetime_index])
        df = pd.concat([df, df_temp])

    return df

# Define the path of the directory
dir_path = "ADD PATH"
  # Add your directory path here

# Create an empty DataFrame
df_total = pd.DataFrame()

# Loop through each file in the directory
for file in os.listdir(dir_path):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        file_path = os.path.join(dir_path, file)
        df = parse_excel(file_path)
        df_total = pd.concat([df_total, df])
        
df_total.sort_index(inplace=True)
        
# Reset index
df_total.reset_index(inplace=True)

df_total = df_total.rename(columns={'index': 'datetime'})

# df_total = df_total.sort_values('index')

# Print the dataframe
print(df_total)


In [ ]:
df_total.to_csv("...")

In [ ]:
df_loaded = pd.read_csv("...")

In [ ]:
df_loaded

In [ ]:
# 3. Apply UTC Convention
# View all timezones using the code below
"""
>>> import pytz
>>> pytz.all_timezones
"""


def convert_to_utc(df, source_timezone):
    # Ensure the DataFrame has a DatetimeIndex
    #     if not isinstance(df.index, pd.DatetimeIndex):
    #         raise ValueError("The DataFrame must have a DatetimeIndex.")

    # Create timezone objects for source and target (UTC) timezones
    source_tz = pytz.timezone(source_timezone)
    target_tz = pytz.UTC

    # Convert the "datetime" column to a DatetimeIndex
    # df['Datetime'] = pd.to_datetime(df['Datetime'])

    if not isinstance(df, pd.DatetimeIndex):
        # Convert the "datetime" column to a DatetimeIndex
        df["datetime"] = pd.to_datetime(df["datetime"])
        df.set_index("datetime", inplace=True)

    # df.set_index('Datetime', inplace=True)

    # Localize the DatetimeIndex to the source timezone, handling ambiguous and non-existent times
    df_source_tz = df.index.tz_localize(source_tz, ambiguous="NaT", nonexistent="NaT")

    # Convert the DatetimeIndex to the target timezone (UTC)
    df_utc = df_source_tz.tz_convert(target_tz)

    # Set the DatetimeIndex as a column in the DataFrame
    df["ts"] = df_utc

    df.set_index("ts", inplace=True)

    # to get rid of the time zone different impliment:
    # Format the DatetimeIndex without the timezone offset
    # df_utc.index = df_utc.index.strftime('%Y-%m-%d %H:%M:%S')

    return df


# orginally got this error: NonExistentTimeError: 2019-03-31 02:00:00, so they dont adjust for daylight savings.


source_timezone = "Europe/Malta"  # Replace with the desired timezone
df_UTC = convert_to_utc(df_loaded, source_timezone)


# will need to createa a lamda/function to go through all of the 15min data to convert it
# to the correct UTC. It would be worth making this into a function so it can be used by the hourly.
# Want it so that i just have to pass a dataframe that has a Datetime funciton into it.

print("-------- COMPLETE 3 -------")

In [ ]:
df_UTC

In [ ]:
df_UTC.to_csv("...")

In [ ]:
df_UTC

In [ ]:
df_UTC.reset_index(inplace=True)

In [ ]:
# df_UTC['ts'] = pd.to_datetime(df_UTC['ts'])
df_UTC['ts'] = df_UTC['ts'].astype('datetime64[ns]')

In [ ]:
df_UTC

In [ ]:
df_UTC = df_UTC.set_index('ts')

In [ ]:
df_UTC

In [ ]:
def pdtocdf(df):
    
    
#     df_UTC.index = pd.to_datetime(df_UTC.index)
#     df.reset_index(drop=False)
    
    data_array = xr.Dataset(df)
    
    data_array = data_array.rename({'pv_data': 'power'})
    
    data_array = data_array.expand_dims({"pv_id": [0]})
    
    data_array = data_array.assign_coords({"latitude": [35]})
    data_array = data_array.assign_coords({"longitude": [15]})

#     data_array = data_array.set_coords("ts").swap_dims({"dim_0": "ts"})

#     data_array = data_array.drop("dim_0")


    # Save the DataArray as a NetCDF file
    data_array.to_netcdf("...")


    return data_array


data_array = pdtocdf(df_UTC)

In [ ]:
data_array